# Read Data

In [1]:
import json
import jsonlines

In [88]:
import os

In [22]:
import openai
import jsonlines
import pickle
import openai
import pandas as pd
from typing import List, Tuple, Text

In [38]:
from tqdm import tqdm

In [6]:
import torch 
from transformers import BertTokenizer, BertModel
from libs.faiss_indexer import DenseFlatIndexer

# Download Weight

In [2]:
from huggingface_hub import hf_hub_download
import joblib

/home/dhnam/.pyenv/versions/3.8.17/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
REPO_ID = 'Nma/DPR_HSC_FAQs'
FILENAME = 'dpr_biencoder.400'
local_dir = "/home/dhnam/SR-DPR/checkpoint/HSC_FAQs"
hf_hub_download(repo_id=REPO_ID, filename=FILENAME, local_dir=local_dir)

'/home/dhnam/SR-DPR/checkpoint/HSC_FAQs/dpr_biencoder.400'

# Load Model

In [158]:
pretrained_model_path = "NlpHUST/vibert4news-base-cased"
# model_path = '/home/dhnam/SR-DPR/checkpoint/HSC_FAQs/dpr_biencoder.400' #New train
# model_path = '/home/dhnam/SR-DPR/checkpoint/general/dpr_biencoder.1000' #General
model_path = '/home/dhnam/SR-DPR/checkpoint/HSC/dpr_biencoder.400' #A loi

In [159]:
use_cuda = True

In [160]:
## Load Tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [161]:
## Load Context Model
saved_state = torch.load(model_path, map_location=lambda s, t: s)
saved_state = saved_state['model_dict']
ctx_prefix = "ctx_model."

ctx_model_state = {
    k[len(ctx_prefix):]: v
    for k, v in saved_state.items()
    if k.startswith(ctx_prefix)
}
# assert len(ctx_model_state) == 200
ctx_model = BertModel.from_pretrained(pretrained_model_path)
ctx_model.load_state_dict(ctx_model_state,  strict=False)
ctx_model.eval()

Some weights of the model checkpoint at NlpHUST/vibert4news-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(62000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [162]:
##Load Question Model 
saved_state = torch.load(model_path, map_location=lambda s, t: s)
saved_state = saved_state['model_dict']
question_prefix = "question_model."
question_model_state = {
        k[len(question_prefix):]: v
        for k, v in saved_state.items()
        if k.startswith(question_prefix)
    }
# assert len(question_model_state) == 200
question_model = BertModel.from_pretrained(pretrained_model_path)
question_model.load_state_dict(question_model_state, strict= False)
question_model.eval()

Some weights of the model checkpoint at NlpHUST/vibert4news-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(62000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [163]:
if use_cuda and torch.cuda.is_available():
    device = 'cuda'
    ctx_model.to('cuda')
    # question_model.to('cuda')
else:
    device ='cpu'

# Load Data

In [164]:
with jsonlines.open('/home/dhnam/SR-DPR/data/hsc_newdata/eval.json') as reader:
    data = [item for item in reader]

In [165]:
base_questions = []
base_contents = []
question_base_questions = []
question_base_contents = []

for idx, item in enumerate(data):
    base_questions.append(item['base_question'])
    base_contents.append(item['base_content'])
    questions = item['questions']
    question_base_question = questions['questionbasequestion']
    question_base_content = questions['questionbasecontent']
    for question in question_base_question:
        question_base_questions.append((idx, question))
    for question in question_base_content:
        question_base_contents.append((idx, question))

In [166]:
file_path = "/home/dhnam/SR-DPR/data/hsc_newdata/Corpus_v2.xlsx"
adding_data = pd.read_excel(file_path, None)
sheet = "FAQs"
adding_data = adding_data.get(sheet)

In [167]:
# adding_dataadding_data = adding_data.fillna(' ')
adding_data = adding_data.dropna(how='any')

In [168]:
for i in adding_data['Answer']:
    if i == ' ':
        continue
    base_contents.append(i)

In [169]:
corpus_index_path = '/home/dhnam/SR-DPR/indexes/eval'

In [170]:
data_to_be_indexed = []
with torch.no_grad():
    for idx, content in tqdm(enumerate(base_contents)):
        all_emb = []
        inputs = tokenizer(content, return_tensors="pt", truncation=True, max_length=512) 
        outputs = ctx_model(input_ids=inputs.input_ids.to(device), attention_mask=inputs.attention_mask.to(device), return_dict=True)
        
        sequence_output = outputs.last_hidden_state # [bsz, seq_len, hidden_size] -> [1, 20, 768]
        pooled_output = sequence_output[:, 0, :]
        all_emb.append(pooled_output)
        all_emb = torch.cat(all_emb, dim=0)
        data = {"article_id": idx, "text": content}
        data_to_be_indexed.append((data,  all_emb[0].cpu().numpy()))

indexer = DenseFlatIndexer()
indexer.init_index(vector_sz=768) #(vector_sz=embeddings[0].shape[0]
indexer.index_data(data_to_be_indexed)
index_path = os.path.dirname(corpus_index_path)
indexer.serialize(index_path)

170it [00:03, 54.60it/s]


In [171]:
Q_TP_top5 = Q_FP_top5 = 0
Q_TP_top3 = Q_FP_top3 = 0

A_TP_top5 = A_FP_top5 = 0
A_TP_top3 = A_FP_top3 = 0

TP_top5 = FP_top5 = 0
TP_top3 = FP_top3 = 0

ori_TP_top5 = ori_FP_top5 = 0
ori_TP_top3 = ori_FP_top3 = 0

In [172]:
for idx, question in enumerate(base_questions):
    query = question
    query_tokens = tokenizer.tokenize(query)

    query_tokens = [tokenizer.cls_token] + query_tokens + [tokenizer.sep_token]
    query_ids = tokenizer.convert_tokens_to_ids(query_tokens)
    outputs = question_model(input_ids=torch.tensor([query_ids]), return_dict=True)
    query_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()

    retrieval_results = indexer.search_knn(query_embedding, top_docs=5)
    metas = retrieval_results[0][0]
    # scores = retrieval_results[0][1]
    # scores = [float(score) for score in scores]
    # retrieval_results = [{**meta, "score": score} for meta, score in zip(metas, scores)]   
    retrieval_id = [x['article_id'] for x in metas]
    if idx in retrieval_id:
        ori_TP_top5 +=1
        TP_top5 += 1
        if idx in retrieval_id[:3]:
            ori_TP_top3+=1
            TP_top3 +=1
    else:
        ori_FP_top5 +=1
        ori_FP_top3 +=1
        FP_top5 += 1
        FP_top3 +=1

In [173]:
for data in tqdm(question_base_questions):
    idx = data[0]
    question = data[1]
    query = question
    query_tokens = tokenizer.tokenize(query)

    query_tokens = [tokenizer.cls_token] + query_tokens + [tokenizer.sep_token]
    query_ids = tokenizer.convert_tokens_to_ids(query_tokens)
    outputs = question_model(input_ids=torch.tensor([query_ids]), return_dict=True)
    query_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()

    retrieval_results = indexer.search_knn(query_embedding, top_docs=5)
    metas = retrieval_results[0][0]
    # scores = retrieval_results[0][1]
    # scores = [float(score) for score in scores]
    # retrieval_results = [{**meta, "score": score} for meta, score in zip(metas, scores)]   
    retrieval_id = [x['article_id'] for x in metas]
    if idx in retrieval_id:
        Q_TP_top5 +=1
        TP_top5 += 1
        if idx in retrieval_id[:3]:
            Q_TP_top3+=1
            TP_top3 +=1
    else:
        Q_FP_top5 +=1
        Q_FP_top3 +=1
        FP_top5 += 1
        FP_top3 +=1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:08<00:00, 16.86it/s]


In [174]:
for data in tqdm(question_base_contents):
    idx = data[0]
    question = data[1]
    query = question
    query_tokens = tokenizer.tokenize(query)

    query_tokens = [tokenizer.cls_token] + query_tokens + [tokenizer.sep_token]
    query_ids = tokenizer.convert_tokens_to_ids(query_tokens)
    outputs = question_model(input_ids=torch.tensor([query_ids]), return_dict=True)
    query_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()

    retrieval_results = indexer.search_knn(query_embedding, top_docs=5)
    metas = retrieval_results[0][0]
    # scores = retrieval_results[0][1]
    # scores = [float(score) for score in scores]
    # retrieval_results = [{**meta, "score": score} for meta, score in zip(metas, scores)]   
    retrieval_id = [x['article_id'] for x in metas]
    if idx in retrieval_id:
        A_TP_top5 +=1
        TP_top5 += 1
        if idx in retrieval_id[:3]:
            A_TP_top3+=1
            TP_top3 +=1
    else:
        A_FP_top5 +=1
        A_FP_top3 +=1
        FP_top5 += 1
        FP_top3 +=1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:09<00:00, 16.25it/s]


In [136]:
## NEw train
print(f"Accuracy Top@3 with original Question: {ori_TP_top3/(ori_TP_top3+ori_FP_top3)}")
print(f"Accuracy Top@5 with original Question: {ori_TP_top5/(ori_TP_top5+ori_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Question: {Q_TP_top3/(Q_TP_top3+Q_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Question: {Q_TP_top5/(Q_TP_top5+Q_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Answer: {A_TP_top3/(A_TP_top3+A_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Answer: {A_TP_top5/(A_TP_top5+A_FP_top5)}")

print(f"Accuracy Top@3 All: {TP_top3/(TP_top3+FP_top3)}")
print(f"Accuracy Top@5 All: {TP_top5/(TP_top5+FP_top5)}")

Accuracy Top@3 with original Question: 0.9148936170212766
Accuracy Top@5 with original Question: 0.92
Accuracy Top@3 with Generated Question with Question: 0.8357142857142857
Accuracy Top@5 with Generated Question with Question: 0.8466666666666667
Accuracy Top@3 with Generated Question with Answer: 0.9166666666666666
Accuracy Top@5 with Generated Question with Answer: 0.92


In [157]:
# general
print(f"Accuracy Top@3 with original Question: {ori_TP_top3/(ori_TP_top3+ori_FP_top3)}")
print(f"Accuracy Top@5 with original Question: {ori_TP_top5/(ori_TP_top5+ori_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Question: {Q_TP_top3/(Q_TP_top3+Q_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Question: {Q_TP_top5/(Q_TP_top5+Q_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Answer: {A_TP_top3/(A_TP_top3+A_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Answer: {A_TP_top5/(A_TP_top5+A_FP_top5)}")

print(f"Accuracy Top@3 All: {TP_top3/(TP_top3+FP_top3)}")
print(f"Accuracy Top@5 All: {TP_top5/(TP_top5+FP_top5)}")

Accuracy Top@3 with original Question: 0.9148936170212766
Accuracy Top@5 with original Question: 0.92
Accuracy Top@3 with Generated Question with Question: 0.903448275862069
Accuracy Top@5 with Generated Question with Question: 0.9066666666666666
Accuracy Top@3 with Generated Question with Answer: 0.9328859060402684
Accuracy Top@5 with Generated Question with Answer: 0.9333333333333333


In [176]:
# A Loi
print(f"Accuracy Top@3 with original Question: {ori_TP_top3/(ori_TP_top3+ori_FP_top3)}")
print(f"Accuracy Top@5 with original Question: {ori_TP_top5/(ori_TP_top5+ori_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Question: {Q_TP_top3/(Q_TP_top3+Q_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Question: {Q_TP_top5/(Q_TP_top5+Q_FP_top5)}")

print(f"Accuracy Top@3 with Generated Question with Answer: {A_TP_top3/(A_TP_top3+A_FP_top3)}")
print(f"Accuracy Top@5 with Generated Question with Answer: {A_TP_top5/(A_TP_top5+A_FP_top5)}")

print(f"Accuracy Top@3 All: {TP_top3/(TP_top3+FP_top3)}")
print(f"Accuracy Top@5 All: {TP_top5/(TP_top5+FP_top5)}")

Accuracy Top@3 with original Question: 0.9565217391304348
Accuracy Top@5 with original Question: 0.96
Accuracy Top@3 with Generated Question with Question: 0.8226950354609929
Accuracy Top@5 with Generated Question with Question: 0.8333333333333334
Accuracy Top@3 with Generated Question with Answer: 0.8321678321678322
Accuracy Top@5 with Generated Question with Answer: 0.84
Accuracy Top@3 All: 0.8454545454545455
Accuracy Top@5 All: 0.8542857142857143
